# Criar uma consulta que mostre, nesta ordem: Nome, Estado e Status dos clientes

In [1]:
from src.spark_client import get_spark

spark = get_spark()


In [ ]:
from pyspark.sql.functions import col, sum, round

df_clients = spark.read.parquet("/data/raw/Clientes.parquet")

df_clients.select(col("Cliente"), col("Estado"), col("Status")).show()

# Criar uma consulta que mostre apenas os clientes do Status "platinum" e "gold"

In [ ]:
df_clients.filter((col("Status") == "Platinum") | (col("Status") == "Gold")).show()

# Demonstrar quanto cada Status de Clientes representa em vendas

In [ ]:
df_sales = spark.read.parquet("/data/raw/Vendas.parquet")

df_joined = df_clients.join(df_sales, on=df_clients.ClienteID == df_sales.ClienteID, how="inner")

df_sales_by_status = df_joined.groupBy("Status").agg(round(sum("Total"), 2).alias("Total por Status"))
df_sales_by_status.show()

# Criar um banco de dados no DW do Spark chamado VendasVarejo e persistir todas as tabelas neste banco de dados

In [6]:
spark.sql("CREATE DATABASE IF NOT EXISTS VendasVarejo")
spark.catalog.setCurrentDatabase("VendasVarejo")

In [10]:
spark.read.parquet("/data/raw/Clientes.parquet").write.saveAsTable("Clientes")
spark.read.parquet("/data/raw/ItensVendas.parquet").write.saveAsTable("ItensVendas")
spark.read.parquet("/data/raw/Produtos.parquet").write.saveAsTable("Produtos")
spark.read.parquet("/data/raw/Vendas.parquet").write.saveAsTable("Vendas")
spark.read.parquet("/data/raw/Vendedores.parquet").write.saveAsTable("Vendedores")